In [1]:
import plotly.express as px
from scrap import *

In [2]:
dropdown_categories = [val for val in list(get_fund_list().keys())]

dropdown_value = dropdown_categories[:3]

In [17]:
data = pd.DataFrame(columns=['Date'])

for val in dropdown_value:
    print(val)

    df = fund_download(val)
    df.columns = ['Date', val]
    data = pd.merge(data, df, how='outer', on='Date')

S&P 500 UCITS ETF
Plotting dataframe...
U.S. Equity Index Fund
Plotting dataframe...
FTSE Developed World ex-U.K. Equity Index Fund
Plotting dataframe...


In [18]:
data

,Date,S&P 500 UCITS ETF,U.S. Equity Index Fund,FTSE Developed World ex-U.K. Equity Index Fund
0,2022-09-23,57.70,741.7042,518.8395
1,2022-09-22,58.71,731.2412,511.9133
2,2022-09-21,59.20,734.8905,514.0917
3,2022-09-20,60.23,742.0716,518.1966
4,2022-09-16,60.50,744.6807,520.3950
...,...,...,...,...
3363,2009-06-29,NaN,102.3487,102.1191
3364,2009-06-26,NaN,101.6697,101.6331
3365,2009-06-25,NaN,103.1402,102.3949
3366,2009-06-24,NaN,99.7198,100.2270


In [25]:
current_year = datetime.now().year
data['year-month'] = data.Date.dt.strftime('%Y-%m')
data = data[((data.Date.dt.year == 2022) | (data['year-month'] == f'{current_year-1}-12'))].groupby('year-month').mean()
data

,S&P 500 UCITS ETF,U.S. Equity Index Fund,FTSE Developed World ex-U.K. Equity Index Fund
year-month,,,
2021-12,72.265000,774.331162,547.067086
2022-01,70.702500,738.255700,526.594175
2022-02,68.663500,718.140575,512.496140
2022-03,68.126087,731.461865,515.880178
2022-04,68.183500,744.408295,525.099247
2022-05,62.882273,708.323610,504.581824
2022-06,60.527273,686.313650,489.611295
2022-07,60.881429,712.377952,502.481943
2022-08,64.857826,761.727986,532.244336


In [43]:
d = pd.DataFrame()
data_prev = data.iloc[1:, :]
for prev, nxt in zip(data.iterrows(), data_prev.iterrows()):
    i, row_prev = prev
    j, row_next = nxt
    
    increase = 100*(row_prev - row_next)/row_prev
    increase.name = j
    d = d.append(increase)
d

,FTSE Developed World ex-U.K. Equity Index Fund,S&P 500 UCITS ETF,U.S. Equity Index Fund
2022-01,3.742304,2.162181,4.658919
2022-02,2.677211,2.883915,2.724683
2022-03,-0.660305,0.782676,-1.854970
2022-04,-1.787056,-0.084275,-1.769939
2022-05,3.907342,7.774942,4.847432
2022-06,2.966918,3.745094,3.107331
2022-07,-2.628748,-0.585118,-3.797725
2022-08,-5.923077,-6.531380,-6.927507
2022-09,1.684660,5.741907,1.533799


In [5]:
data[data.columns[1]].diff()

0        NaN
1      -0.22
2       2.77
3      -0.67
4      -0.96
        ... 
3357     NaN
3358     NaN
3359     NaN
3360     NaN
3361     NaN
Name: S&P 500 UCITS ETF, Length: 3362, dtype: float64

In [6]:
def calculate_rsi(data, period=13):
    d = pd.DataFrame()

    for col in data.columns[1:]:
        net_change = data[col].diff()
        increase = net_change.clip(lower=0)
        decrease = -1*net_change.clip(upper=0)
        ema_up = increase.ewm(com=period, adjust=False).mean()
        ema_down = decrease.ewm(com=period, adjust=False).mean()
        RS = ema_up/ema_down
        d[f'RSI_{col}'] = 100 - (100/(1+RS)) 
    return d

In [9]:
d = calculate_rsi(data)

In [10]:
# data = data.dropna().reset_index(drop=True)
d

,RSI_S&P 500 UCITS ETF,RSI_U.S. Equity Index Fund,RSI_FTSE Developed World ex-U.K. Equity Index Fund
0,NaN,NaN,NaN
1,0.000000,0.000000,100.000000
2,49.200710,81.556986,100.000000
3,43.611481,81.404649,97.150688
4,37.106938,64.559309,84.855922
...,...,...,...
3357,44.726724,41.852920,38.408577
3358,44.726724,39.583783,36.546750
3359,44.726724,46.365770,41.346254
3360,44.726724,36.189611,33.565546


In [33]:
idx = 4
fig = px.line(x=data.Date, y=data[data.columns[idx]], title=data.columns[idx])
fig.add_hline(y=20, line_dash="dot")
fig.add_hline(y=60, line_dash="dot")

In [34]:
import datetime

dd = pd.DataFrame(columns=['Current price', 
                      'Profit (compared to yesterday)', 
                      'Profit (compared to last month)',
                     'Profit (compared to last 6 months)',
                    'Profit (compared to last year)',
                    'Profit (compared to last 5 years)',
            'Profit (compared to last 10 years)'],
                 index=data.columns[1:])

for fund in data.columns[1:]:
    try:
        p_today = data[fund].iloc[0]
        dd.loc[fund, 'Current price'] = p_today

        p_yesterday = data[fund].iloc[1]
        dd.loc[fund, 'Profit (compared to yesterday)'] = round(100*(p_today-p_yesterday)/p_yesterday, 2)

        p_month = data[fund].iloc[5*4]
        dd.loc[fund, 'Profit (compared to last month)'] = round(100*(p_today - p_month)/p_month, 2)

        p_6m = data[fund].iloc[5*4*6]
        dd.loc[fund, 'Profit (compared to last 6 months)'] = round(100*(p_today - p_6m)/p_6m, 2)

        p_year = data[fund].iloc[5*4*12]
        dd.loc[fund, 'Profit (compared to last year)'] = round(100*(p_today - p_year)/p_year, 2)

        p_5y =  data[fund].iloc[5*4*12*5]
        dd.loc[fund, 'Profit (compared to last 5 years)'] = round(100*(p_today - p_5y)/p_5y, 2)

        p_10y =  data[fund].iloc[5*4*12*10]
        dd.loc[fund, 'Profit (compared to last 10 years)'] = round(100*(p_today - p_10y)/p_10y, 2)


    except Exception as e:
        print(e)

dd

,Current price,Profit (compared to yesterday),Profit (compared to last month),Profit (compared to last 6 months),Profit (compared to last year),Profit (compared to last 5 years),Profit (compared to last 10 years)
S&P 500 UCITS ETF,64.18,1.05,-4.24,-8.96,-4.71,NaN,NaN
U.S. Equity Index Fund,771.934,0.01,-1.26,2.27,7.41,80.34,276.11
FTSE Developed World ex-U.K. Equity Index Fund,537.114,0.19,-1.35,1.17,5.33,59.15,201.97
RSI_S&P 500 UCITS ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RSI_U.S. Equity Index Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RSI_FTSE Developed World ex-U.K. Equity Index Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_perc = data.groupby(data.Date.dt.year).mean().reset_index()
df_perc

,Date,S&P 500 UCITS ETF,U.S. Equity Index Fund,FTSE Developed World ex-U.K. Equity Index Fund,FTSE All-World UCITS ETF
0,2009,NaN,116.315897,116.492224,NaN
1,2010,NaN,138.944660,133.215630,NaN
2,2011,NaN,152.387439,141.366448,NaN
3,2012,NaN,169.403312,147.801441,NaN
4,2013,NaN,209.545849,180.501749,NaN
5,2014,NaN,237.434190,197.961091,NaN
6,2015,NaN,276.207400,221.972795,NaN
7,2016,NaN,320.895342,251.603944,NaN
8,2017,NaN,399.741331,313.771344,NaN
9,2018,NaN,439.948975,337.714296,NaN


In [6]:
dd = df_perc.copy()
for future, past in zip(df_perc.iloc[1:].iterrows(), df_perc.iterrows()):
    i, row_f = future
    j, row_p = past
    for col in df_perc.columns[1:]:
        profit = round(100*(row_f[col] - row_p[col])/ row_p[col], 2)
        dd.loc[j, col] = profit
dd = dd.iloc[:-1]
dd

,Date,S&P 500 UCITS ETF,U.S. Equity Index Fund,FTSE Developed World ex-U.K. Equity Index Fund,FTSE All-World UCITS ETF
0,2009,NaN,19.45,14.36,NaN
1,2010,NaN,9.67,6.12,NaN
2,2011,NaN,11.17,4.55,NaN
3,2012,NaN,23.70,22.12,NaN
4,2013,NaN,13.31,9.67,NaN
5,2014,NaN,16.33,12.13,NaN
6,2015,NaN,16.18,13.35,NaN
7,2016,NaN,24.57,24.71,NaN
8,2017,NaN,10.06,7.63,NaN
9,2018,NaN,11.61,9.05,NaN


In [18]:
px.imshow(dd.iloc[:, 1:].corr())

In [8]:
# d = dd.melt(id_vars=['Date']).dropna()
# d

In [9]:
# d['color'] = 'blue'
# d.loc[d['value'] < 0, 'color'] = 'red'
# d

In [11]:
# # df = px.data.tips()
# fig = px.bar(d, x="Date", y="value", facet_col="variable")
# fig.show()